In [1]:
import pandas as pd
import json
from pymongo import MongoClient
from src.geopoint_mongo_functions import geoQueryNear
from src.geopoint_mongo_functions import queryMongo
from src.geopoint_mongo_functions import creaMarkerenMapa
from src.geopoint_mongo_functions import rankingByAttribute
from src.geopoint_mongo_functions import rankingByDistance
from folium import Map, FeatureGroup, LayerControl, CircleMarker
from folium.plugins import FastMarkerCluster 
import math


# Mongodb geospartial with near

In [2]:
# importar el Database desde MongoDB
dbName = "Italian_Companies"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [3]:
# lanzar query por cada categoría para devolver las tablas con las informacíones
# la query es así para devolver toda la tabla. query = {"name":{"$ne": None}}
query = {"name":{"$ne": None}}
projection = {"_id":0}
clean_db_aeropuertos = queryMongo(db, 'clean_db_aeropuertos', query, projection)
clean_db_aeropuertos.head(5)

,name,address,LAT_WGS84,LONG_WGS84,geopoint
0,Aeroporto di Milano Linate (LIN) (Aeroporto di...,Viale Enrico Forlanini,45.460409,9.279444,"{'type': 'Point', 'coordinates': [9.2794441997..."
1,Aeroporto di Bresso (LIMB),Via Antonio Gramsci 1,45.537930,9.198891,"{'type': 'Point', 'coordinates': [9.198891273,..."
2,"Aeroporto di Orio al Serio ""Il Caravaggio"" (BG...",Via Aeroporto 13,45.665874,9.698644,"{'type': 'Point', 'coordinates': [9.6986441614..."
3,Aeroporto di Milano Malpensa (MXP) (Aeroporto ...,Superstrada Malpensa 2000,45.629810,8.717244,"{'type': 'Point', 'coordinates': [8.7172436714..."
4,"Aeroporto di Cameri ""Natale e Silvio Palli"" (L...",Loc. Aeroporto,45.539755,8.655301,"{'type': 'Point', 'coordinates': [8.6553011087..."


In [4]:
# con folium visulizár los datos
# en este caso los aeropuertos cerca e Milan.
# los feature group permite devolver la info en diferente layers 
map_ = Map(location = [45.464150, 9.187716],
                  tiles = 'stamentoner', 
                  zoom_start = 10)

feature_group = FeatureGroup(name='Aeropuertos')
aeropuertos_table = pd.DataFrame(clean_db_aeropuertos)
        
creaMarkerenMapa(aeropuertos_table, '#3186cc', 'LAT_WGS84', 'LONG_WGS84', ["name", "address"], feature_group)

feature_group.add_to(map_)
LayerControl().add_to(map_)
map_

In [5]:
# mismo procedimiento 
query = {"name":{"$ne": None}}
projection = {"_id":0}
clean_db_basketballs = queryMongo(db, 'clean_db_basketballs', query, projection)
clean_db_basketballs.head(5)

,name,address,LAT_WGS84,LONG_WGS84,geopoint
0,Knights Palace,Via Parma 7,45.581819,8.894455,"{'type': 'Point', 'coordinates': [8.8944551989..."
1,PalaCandy,Viale Stucchi 16,45.585332,9.309250,"{'type': 'Point', 'coordinates': [9.3092497825..."
2,Palasport Pianella,"Via Cantù, 10",45.740063,9.100072,"{'type': 'Point', 'coordinates': [9.1000717969..."
3,Olimpia Pallacanestro Milano,None,45.480547,9.142221,"{'type': 'Point', 'coordinates': [9.1422211705..."
4,Enerxenia Arena,Piazzale Antonio Gramsci 1,45.835943,8.802508,"{'type': 'Point', 'coordinates': [8.802507805,..."


In [6]:
# mismo procedimiento. SLa funcion creaMarkerenMapa nos permite poner los Marker en cada lugar 
# que aparece en  la tabla
map_ = Map(location = [45.464150, 9.187716],
                  tiles = 'stamentoner', 
                  zoom_start = 14)

feature_group = FeatureGroup(name='Basketball Stadium')
basketball_table = pd.DataFrame(clean_db_basketballs)
creaMarkerenMapa(basketball_table, '#46dbac', 'LAT_WGS84', 'LONG_WGS84', ["name", "address"], feature_group)

In [7]:
# mismo procedimiento
query = {"name":{"$ne": None}}
projection = {"_id":0}
clean_db_design = queryMongo(db, 'new_design', query, projection)
clean_db_design.head(5)

,name,address,LAT_WGS84,LONG_WGS84,geopoint
0,10 Corso Como,Corso Como 10,45.481852,9.187831,"{'type': 'Point', 'coordinates': [9.1878313944..."
1,Galleria Carla Sozzani,Corso Como 10,45.481846,9.187826,"{'type': 'Point', 'coordinates': [9.1878260613..."
2,Fratelli Radice S.r.l.,Via Sprelunga 4,45.649591,9.132403,"{'type': 'Point', 'coordinates': [9.1324026042..."
3,Spazio Rossana Orlandi,Via Matteo Bandello 14-16,45.463812,9.166722,"{'type': 'Point', 'coordinates': [9.1667218642..."
4,Echi di Carta,Via Vetta d'Italia 24,45.607628,9.242658,"{'type': 'Point', 'coordinates': [9.2426583872..."


In [8]:
 feature_group2 = FeatureGroup(name='Design Company')
design_table = pd.DataFrame(clean_db_design)
creaMarkerenMapa(design_table, '#edd05c', 'LAT_WGS84', 'LONG_WGS84', ["name", "address"], feature_group2)

In [9]:
query = {"LONG_WGS84":{"$ne": 'null'}}
projection = {"_id":0}
clean_db_sitios_para_diversion = queryMongo(db, 'clean_sitios_para_diversion', query, projection)
clean_db_sitios_para_diversion.head(5)

,insegna,LONG_WGS84,LAT_WGS84,geopoint
0,PERSEFONE,9.175731,45.496658,"{'type': 'Point', 'coordinates': [9.1757310253..."
1,OFFICINA 12,9.175366,45.452087,"{'type': 'Point', 'coordinates': [9.1753664722..."
2,MOGAMO,9.208940,45.485067,"{'type': 'Point', 'coordinates': [9.2089403531..."
3,NOY CONCEPT RESTAURANT CAFE',9.162191,45.465265,"{'type': 'Point', 'coordinates': [9.1621909245..."
4,PIZZERIA DI PORTA GARIBALDI,9.187345,45.481580,"{'type': 'Point', 'coordinates': [9.1873446147..."


In [10]:
feature_group3 = FeatureGroup(name='Sitios para diversion')
sitios_para_diversion_table = pd.DataFrame(clean_db_sitios_para_diversion)
creaMarkerenMapa(sitios_para_diversion_table, '#ff8957', 'LAT_WGS84', 'LONG_WGS84', ["insegna", 'geopoint'], feature_group3)

In [11]:
query = {"name":{"$ne": None}}
projection = {"_id":0}
clean_db_starbucks_table = queryMongo(db, 'starbucks_table', query, projection)
clean_db_starbucks_table.head(5)

,name,location,geopoint
0,Starbucks Vercelli,"Piazzale Francesco Baracca, 10 angolo Corso Ve...","{'type': 'Point', 'coordinates': [9.16487, 45...."
1,Starbucks Malpensa,Aeroporto Milano Malpensa T1 - Partenze Ferno ...,"{'type': 'Point', 'coordinates': [9.11615, 45...."
2,Starbucks Garibaldi,"Corso Garibaldi, 118 Milano 20121","{'type': 'Point', 'coordinates': [9.18336, 45...."
3,Starbucks Turati,"Via Filippo Turati, 25 Milano 20121","{'type': 'Point', 'coordinates': [9.1949, 45.4..."
4,Starbucks Durini,"Via Durini, 28 Milan 20121","{'type': 'Point', 'coordinates': [9.19836, 45...."


In [12]:
feature_group4 = FeatureGroup(name='Starbucks')
for i,row in clean_db_starbucks_table.iterrows():
    CircleMarker((row['geopoint']['coordinates'][1], row['geopoint']['coordinates'][0]),
                         color='#f739f4',
                         fill_color='#f739f4',
                         popup = (f'{row["name"]}{row["location"]}')
                         ).add_to(feature_group4)
feature_group.add_to(map_)
feature_group2.add_to(map_)
feature_group3.add_to(map_)
feature_group4.add_to(map_)
LayerControl().add_to(map_)
map_

# Buscar el lugar favorito

In [13]:
query = {"_id":{"$ne": None}}
projection = {"_id":0}
clean_db_civicos_Milan = queryMongo(db, 'clean_civicos_milan', query, projection)
clean_db_civicos_Milan.head(5)

,NUMEROCOMPLETO,TIPO,DESCRITTIVO,LONG_WGS84,LAT_WGS84,geopoint
0,21N08,Piazza,DEL DUOMO,9.189762,45.464840,"{'type': 'Point', 'coordinates': [9.189761993,..."
1,21,Piazza,DEL DUOMO,9.189584,45.464843,"{'type': 'Point', 'coordinates': [9.1895837028..."
2,19N07,Piazza,DEL DUOMO,9.189030,45.464846,"{'type': 'Point', 'coordinates': [9.1890299669..."
3,19N05,Piazza,DEL DUOMO,9.188923,45.464845,"{'type': 'Point', 'coordinates': [9.1889226416..."
4,19N06,Piazza,DEL DUOMO,9.188972,45.464846,"{'type': 'Point', 'coordinates': [9.1889721118..."


In [14]:
# creo una nuova tabla con solo las infos que me interesán
final_db_civicos_Milan = clean_db_civicos_Milan[['TIPO', 'DESCRITTIVO', 'NUMEROCOMPLETO','LONG_WGS84', 'LAT_WGS84']]

In [15]:
# la visualizo
final_db_civicos_Milan

,TIPO,DESCRITTIVO,NUMEROCOMPLETO,LONG_WGS84,LAT_WGS84
0,Piazza,DEL DUOMO,21N08,9.189762,45.464840
1,Piazza,DEL DUOMO,21,9.189584,45.464843
2,Piazza,DEL DUOMO,19N07,9.189030,45.464846
3,Piazza,DEL DUOMO,19N05,9.188923,45.464845
4,Piazza,DEL DUOMO,19N06,9.188972,45.464846
...,...,...,...,...,...
63512,Tangenziale,TANGENZIALE EST,46/67,9.258961,45.506749
63513,Via,PIER PAOLO PASOLINI,19P02,9.099759,45.512006
63514,Via,PIER PAOLO PASOLINI,84P01,9.104462,45.508987
63515,Via,PIER PAOLO PASOLINI,15,9.101357,45.511007


In [16]:
# defino por cada collection de Mongo, objeto de mis queries, un radium de busqueda.
# la lista de resume_collections, tanto que la de new_columns me permite lanzar la funcíon 
# rankingByAttribute con un bucle
resume_collections = [{'clean_db_aeropuertos': 10000, 
                      'clean_db_basketballs': 2000,
                      'clean_sitios_para_diversion': 200,
                      'new_design': 1000,
                      'starbucks_table': 1000}]

new_columns = ['AEROPUERTOS 10Km', 
               'BASKETBALL STADIUM 2Km', 
               'SITIOS PARA DIVERSION 500m',
               'DESIGN COMPANIES 500m',
               'STARBUCKS 1000m']

ponderacíon = [0.3, 0.3, 0.15, 0.10, 0.15]

In [17]:
# llamo la funcíon rankingByAttribute 
#%%time
rankingByAttribute(db, clean_db_civicos_Milan, resume_collections, final_db_civicos_Milan, new_columns, ponderacíon)

clean_db_aeropuertos
63517
clean_db_basketballs
63517
clean_sitios_para_diversion
63517
new_design
63517
starbucks_table
63517


,TIPO,DESCRITTIVO,NUMEROCOMPLETO,LONG_WGS84,LAT_WGS84,AEROPUERTOS 10Km,BASKETBALL STADIUM 2Km,SITIOS PARA DIVERSION 500m,DESIGN COMPANIES 500m,STARBUCKS 1000m
0,Piazza,DEL DUOMO,21N08,9.189762,45.464840,0.6,0.3,0.15,1.6,0.45
1,Piazza,DEL DUOMO,21,9.189584,45.464843,0.6,0.3,0.15,1.6,0.45
2,Piazza,DEL DUOMO,19N07,9.189030,45.464846,0.6,0.3,0.15,1.6,0.45
3,Piazza,DEL DUOMO,19N05,9.188923,45.464845,0.6,0.3,0.15,1.6,0.45
4,Piazza,DEL DUOMO,19N06,9.188972,45.464846,0.6,0.3,0.15,1.6,0.45
...,...,...,...,...,...,...,...,...,...,...
63512,Tangenziale,TANGENZIALE EST,46/67,9.258961,45.506749,0.6,0.0,0.00,0.0,0.00
63513,Via,PIER PAOLO PASOLINI,19P02,9.099759,45.512006,0.3,0.0,0.00,0.0,0.00
63514,Via,PIER PAOLO PASOLINI,84P01,9.104462,45.508987,0.3,0.0,0.00,0.1,0.00
63515,Via,PIER PAOLO PASOLINI,15,9.101357,45.511007,0.3,0.0,0.00,0.0,0.00


In [18]:
# por cada direccíon detecta cuantos puntos de intéres hay, y luego se moltiplica con un
# factor de ponderaccíon. El total es la columna que me interesa, y que me devuelve los mejores lugares
final_db_civicos_Milan.head(5)

,TIPO,DESCRITTIVO,NUMEROCOMPLETO,LONG_WGS84,LAT_WGS84,AEROPUERTOS 10Km,BASKETBALL STADIUM 2Km,SITIOS PARA DIVERSION 500m,DESIGN COMPANIES 500m,STARBUCKS 1000m
0,Piazza,DEL DUOMO,21N08,9.189762,45.464840,0.6,0.3,0.15,1.6,0.45
1,Piazza,DEL DUOMO,21,9.189584,45.464843,0.6,0.3,0.15,1.6,0.45
2,Piazza,DEL DUOMO,19N07,9.189030,45.464846,0.6,0.3,0.15,1.6,0.45
3,Piazza,DEL DUOMO,19N05,9.188923,45.464845,0.6,0.3,0.15,1.6,0.45
4,Piazza,DEL DUOMO,19N06,9.188972,45.464846,0.6,0.3,0.15,1.6,0.45


In [19]:
final_db_civicos_Milan['TOTAL'] = final_db_civicos_Milan['AEROPUERTOS 10Km'] + final_db_civicos_Milan['BASKETBALL STADIUM 2Km'] + final_db_civicos_Milan['SITIOS PARA DIVERSION 500m'] + final_db_civicos_Milan['DESIGN COMPANIES 500m']  + final_db_civicos_Milan['STARBUCKS 1000m']

In [20]:
# los resultados tienen valores muy cerca, debido al echo que cada edificío de Milan está georeferenciado.
# Muchos efificíos en proximidad tendŕan la misma puntuaccíon
final_db_civicos_Milan.sort_values('TOTAL', ascending= False).head(15)

,TIPO,DESCRITTIVO,NUMEROCOMPLETO,LONG_WGS84,LAT_WGS84,AEROPUERTOS 10Km,BASKETBALL STADIUM 2Km,SITIOS PARA DIVERSION 500m,DESIGN COMPANIES 500m,STARBUCKS 1000m,TOTAL
5682,Viale,MONTE GRAPPA,22N06,9.187883,45.480966,0.6,0.0,2.70,0.7,0.30,4.30
5900,Corso,COMO,11,9.187213,45.482189,0.6,0.0,3.00,0.5,0.15,4.25
5926,Corso,COMO,11N07,9.187228,45.482231,0.6,0.0,3.00,0.5,0.15,4.25
5914,Corso,COMO,9N10,9.187118,45.481895,0.6,0.0,2.85,0.5,0.30,4.25
5913,Corso,COMO,9N08,9.187096,45.481825,0.6,0.0,2.85,0.5,0.30,4.25
5940,Corso,COMO,9,9.187093,45.481790,0.6,0.0,2.85,0.5,0.30,4.25
5908,Corso,COMO,9N05,9.187044,45.481697,0.6,0.0,2.70,0.6,0.30,4.20
5912,Corso,COMO,9N07,9.187068,45.481760,0.6,0.0,2.70,0.6,0.30,4.20
5347,Piazza,VENTICINQUE APRILE,16A,9.187432,45.480838,0.6,0.0,2.55,0.7,0.30,4.15
5685,Viale,MONTE GRAPPA,22N03,9.188044,45.480958,0.6,0.0,2.55,0.7,0.30,4.15


In [21]:
top_3_places = final_db_civicos_Milan.sort_values('TOTAL', ascending= False).head(3)

In [22]:
top_3_places

,TIPO,DESCRITTIVO,NUMEROCOMPLETO,LONG_WGS84,LAT_WGS84,AEROPUERTOS 10Km,BASKETBALL STADIUM 2Km,SITIOS PARA DIVERSION 500m,DESIGN COMPANIES 500m,STARBUCKS 1000m,TOTAL
5682,Viale,MONTE GRAPPA,22N06,9.187883,45.480966,0.6,0.0,2.7,0.7,0.30,4.30
5900,Corso,COMO,11,9.187213,45.482189,0.6,0.0,3.0,0.5,0.15,4.25
5926,Corso,COMO,11N07,9.187228,45.482231,0.6,0.0,3.0,0.5,0.15,4.25


In [23]:
# devuelvo los 3 lugaer top en la mapa
map_ = Map(location = [45.480966, 9.187883],
                  tiles = 'stamentoner', 
                  zoom_start = 16)

feature_group = FeatureGroup(name='Best Place to open the new company')
aeropuertos_table = pd.DataFrame(clean_db_aeropuertos)
        
creaMarkerenMapa(top_3_places, '#3186cc', 'LAT_WGS84', 'LONG_WGS84', ["DESCRITTIVO", "NUMEROCOMPLETO"], feature_group)

feature_group.add_to(map_)
LayerControl().add_to(map_)
map_

In [24]:
distance_db_civicos_Milan = final_db_civicos_Milan

In [25]:
# entento entoncés hacer otra analisís, y por cada edificio entento buscar por cada categoría el lugar mas cercano.
# con esta analisi los resultados seŕan diferentes, pero el hecho de calcular la distancía en metros me permite
# tener mas variancia en los resultados.
# Transoformar lat y long y medír la distancía en metros entre dos puntos:
# The answers above are based on the Haversine formula, which assumes the earth is a sphere, which results in 
# errors of up to about 0.5% (according to help(geopy.distance)). Vincenty distance uses more accurate 
#ellipsoidal models such as WGS-84, and is implemented in geopy. For example,
# import geopy.distance

#coords_1 = (52.2296756, 21.0122287)
#coords_2 = (52.406374, 16.9251681)
#print geopy.distance.vincenty(coords_1, coords_2).km
# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
new_columns2 = ['DIST AEROPUERTOS', 
               'DIST BASKETBALL STADIUM', 
               'DIST SITIOS PARA DIVERSION',
               'DIST DESIGN COMPANIES',
               'DIST STARBUCKS']
rankingByDistance(db, clean_db_civicos_Milan, resume_collections, distance_db_civicos_Milan, new_columns2, ponderacíon)

0
63517
1
63517
2
63517
3
63517
4
63517


,TIPO,DESCRITTIVO,NUMEROCOMPLETO,LONG_WGS84,LAT_WGS84,AEROPUERTOS 10Km,BASKETBALL STADIUM 2Km,SITIOS PARA DIVERSION 500m,DESIGN COMPANIES 500m,STARBUCKS 1000m,TOTAL,DIST AEROPUERTOS,DIST BASKETBALL STADIUM,DIST SITIOS PARA DIVERSION,DIST DESIGN COMPANIES,DIST STARBUCKS
0,Piazza,DEL DUOMO,21N08,9.189762,45.464840,0.6,0.3,0.15,1.6,0.45,3.1,7.031287,0.880205,0.124045,0.152449,0.278891
1,Piazza,DEL DUOMO,21,9.189584,45.464843,0.6,0.3,0.15,1.6,0.45,3.1,7.045220,0.890987,0.137279,0.166372,0.264947
2,Piazza,DEL DUOMO,19N07,9.189030,45.464846,0.6,0.3,0.15,1.6,0.45,3.1,7.088444,0.924540,0.179229,0.197860,0.221686
3,Piazza,DEL DUOMO,19N05,9.188923,45.464845,0.6,0.3,0.15,1.6,0.45,3.1,7.096811,0.931045,0.187461,0.193597,0.213312
4,Piazza,DEL DUOMO,19N06,9.188972,45.464846,0.6,0.3,0.15,1.6,0.45,3.1,7.092958,0.928075,0.183654,0.195574,0.217169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63512,Tangenziale,TANGENZIALE EST,46/67,9.258961,45.506749,0.6,0.0,0.00,0.0,0.00,0.6,5.393495,NaN,NaN,NaN,NaN
63513,Via,PIER PAOLO PASOLINI,19P02,9.099759,45.512006,0.3,0.0,0.00,0.0,0.00,0.3,8.263165,NaN,NaN,NaN,NaN
63514,Via,PIER PAOLO PASOLINI,84P01,9.104462,45.508987,0.3,0.0,0.00,0.1,0.00,0.4,8.048167,NaN,NaN,0.983549,NaN
63515,Via,PIER PAOLO PASOLINI,15,9.101357,45.511007,0.3,0.0,0.00,0.0,0.00,0.3,8.186268,NaN,NaN,NaN,NaN


In [26]:
distance_db_civicos_Milan

,TIPO,DESCRITTIVO,NUMEROCOMPLETO,LONG_WGS84,LAT_WGS84,AEROPUERTOS 10Km,BASKETBALL STADIUM 2Km,SITIOS PARA DIVERSION 500m,DESIGN COMPANIES 500m,STARBUCKS 1000m,TOTAL,DIST AEROPUERTOS,DIST BASKETBALL STADIUM,DIST SITIOS PARA DIVERSION,DIST DESIGN COMPANIES,DIST STARBUCKS
0,Piazza,DEL DUOMO,21N08,9.189762,45.464840,0.6,0.3,0.15,1.6,0.45,3.1,7.031287,0.880205,0.124045,0.152449,0.278891
1,Piazza,DEL DUOMO,21,9.189584,45.464843,0.6,0.3,0.15,1.6,0.45,3.1,7.045220,0.890987,0.137279,0.166372,0.264947
2,Piazza,DEL DUOMO,19N07,9.189030,45.464846,0.6,0.3,0.15,1.6,0.45,3.1,7.088444,0.924540,0.179229,0.197860,0.221686
3,Piazza,DEL DUOMO,19N05,9.188923,45.464845,0.6,0.3,0.15,1.6,0.45,3.1,7.096811,0.931045,0.187461,0.193597,0.213312
4,Piazza,DEL DUOMO,19N06,9.188972,45.464846,0.6,0.3,0.15,1.6,0.45,3.1,7.092958,0.928075,0.183654,0.195574,0.217169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63512,Tangenziale,TANGENZIALE EST,46/67,9.258961,45.506749,0.6,0.0,0.00,0.0,0.00,0.6,5.393495,NaN,NaN,NaN,NaN
63513,Via,PIER PAOLO PASOLINI,19P02,9.099759,45.512006,0.3,0.0,0.00,0.0,0.00,0.3,8.263165,NaN,NaN,NaN,NaN
63514,Via,PIER PAOLO PASOLINI,84P01,9.104462,45.508987,0.3,0.0,0.00,0.1,0.00,0.4,8.048167,NaN,NaN,0.983549,NaN
63515,Via,PIER PAOLO PASOLINI,15,9.101357,45.511007,0.3,0.0,0.00,0.0,0.00,0.3,8.186268,NaN,NaN,NaN,NaN


In [27]:
distance_db_civicos_Milan['TOTAL DISTANCE'] = final_db_civicos_Milan['DIST AEROPUERTOS'] + final_db_civicos_Milan['DIST BASKETBALL STADIUM'] + final_db_civicos_Milan['DIST SITIOS PARA DIVERSION'] + final_db_civicos_Milan['DIST DESIGN COMPANIES'] + final_db_civicos_Milan['DIST STARBUCKS']

In [28]:
top_3_places_distance =distance_db_civicos_Milan.sort_values('TOTAL DISTANCE').head(3)

In [29]:
top_3_places_distance

,TIPO,DESCRITTIVO,NUMEROCOMPLETO,LONG_WGS84,LAT_WGS84,AEROPUERTOS 10Km,BASKETBALL STADIUM 2Km,SITIOS PARA DIVERSION 500m,DESIGN COMPANIES 500m,STARBUCKS 1000m,TOTAL,DIST AEROPUERTOS,DIST BASKETBALL STADIUM,DIST SITIOS PARA DIVERSION,DIST DESIGN COMPANIES,DIST STARBUCKS,TOTAL DISTANCE
1414,Largo,AUGUSTO,7,9.198350,45.463469,0.6,0.3,0.3,1.1,0.45,2.75,6.351559,0.426007,0.001054,0.065149,0.240139,7.083907
1394,Via,CERVA,4,9.198308,45.463672,0.6,0.3,0.3,1.1,0.45,2.75,6.356023,0.448514,0.022766,0.058548,0.217618,7.103468
1393,Via,CERVA,3,9.198149,45.463667,0.6,0.3,0.3,1.1,0.45,2.75,6.368441,0.447952,0.027455,0.046064,0.218805,7.108716


In [30]:
# los 3 sitios tienen un ubicacíon diferente, pero se encuentran siempre en la zona central de Milan
# hemos devuelto dos hipotesys, ahora el CEO de la empresa tiene toda las informaccíon para elegír el mejor sitio.
map_ = Map(location = [45.463469, 9.198350],
                  tiles = 'stamentoner', 
                  zoom_start = 16)

feature_group = FeatureGroup(name='Best Place to open the new company')
aeropuertos_table = pd.DataFrame(clean_db_aeropuertos)
        
creaMarkerenMapa(top_3_places_distance, '#3186cc', 'LAT_WGS84', 'LONG_WGS84', ["DESCRITTIVO", "NUMEROCOMPLETO"], feature_group)

feature_group.add_to(map_)
LayerControl().add_to(map_)
map_